# City and Weather API's

## City API

In [2]:
import pandas as pd
import os
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak'
path = local_project_path+'\\my_booking\\my_booking'
print(path)
os.chdir(path)
chemin_fichier_csv = "./city.csv"
df = pd.read_csv(chemin_fichier_csv, header=None, names=['city','nan'])
cities = df['city'].str.replace(' ', '-').tolist()

list(cities)

C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\my_booking\my_booking


['Mont-Saint-Michel',
 'St-Malo',
 'Bayeux',
 'Le-Havre',
 'Rouen',
 'Paris',
 'Amiens',
 'Lille',
 'Strasbourg',
 'Chateau-du-Haut-Koenigsbourg',
 'Colmar',
 'Eguisheim',
 'Besancon',
 'Dijon',
 'Annecy',
 'Grenoble',
 'Lyon',
 'Gorges-du-Verdon',
 'Bormes-les-Mimosas',
 'Cassis',
 'Marseille',
 'Aix-en-Provence',
 'Avignon',
 'Uzes',
 'Nimes',
 'Aigues-Mortes',
 'Saintes-Maries-de-la-mer',
 'Collioure',
 'Carcassonne',
 'Ariege',
 'Toulouse',
 'Montauban',
 'Biarritz',
 'Bayonne',
 'La-Rochelle']

r.json()[0]  car il y a plusieurs villes avec le même nom, celle qui apparait en premier dans la liste (car c'est une liste) elle celle avec la clef 'importance' la plus élévée

In [3]:
import requests

api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:
    city_req = city.replace(' ', '+')
#    print(city)
    payload = {'city' : city_req,
            'limit' : 1,
            'country' : 'france',
            'country-code' : 'fr',
            'format' : 'json'}
    
    info = requests.get(api_url, 
                          params=payload)
    
#     if info.status_code != 200:
#        print('oops erreur', info.status_code, "pour la ville", city)
#     else:
#        print('city', city, 'latitude', float(info.json()[0]['lat']))


    enriched_cities.append({'city': city,
                            'lat' : float(info.json()[0]['lat']),
                            'lon' : float(info.json()[0]['lon'])})
    
display(enriched_cities)


[{'city': 'Mont-Saint-Michel', 'lat': 48.6359541, 'lon': -1.511459954959514},
 {'city': 'St-Malo', 'lat': 48.649518, 'lon': -2.0260409},
 {'city': 'Bayeux', 'lat': 49.2764624, 'lon': -0.7024738},
 {'city': 'Le-Havre', 'lat': 49.4938975, 'lon': 0.1079732},
 {'city': 'Rouen', 'lat': 49.4404591, 'lon': 1.0939658},
 {'city': 'Paris', 'lat': 48.8534951, 'lon': 2.3483915},
 {'city': 'Amiens', 'lat': 49.8941708, 'lon': 2.2956951},
 {'city': 'Lille', 'lat': 50.6365654, 'lon': 3.0635282},
 {'city': 'Strasbourg', 'lat': 48.584614, 'lon': 7.7507127},
 {'city': 'Chateau-du-Haut-Koenigsbourg', 'lat': 48.2495226, 'lon': 7.3454923},
 {'city': 'Colmar', 'lat': 48.0777517, 'lon': 7.3579641},
 {'city': 'Eguisheim', 'lat': 48.0447968, 'lon': 7.3079618},
 {'city': 'Besancon', 'lat': 47.2380222, 'lon': 6.0243622},
 {'city': 'Dijon', 'lat': 47.3215806, 'lon': 5.0414701},
 {'city': 'Annecy', 'lat': 45.8992348, 'lon': 6.1288847},
 {'city': 'Grenoble', 'lat': 45.1875602, 'lon': 5.7357819},
 {'city': 'Lyon', 'l

In [4]:
df = pd.DataFrame.from_dict(enriched_cities)
#df['size'] = df['city'].apply(lambda x : 10)
display(df)


,city,lat,lon
0,Mont-Saint-Michel,48.635954,-1.511460
1,St-Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le-Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966
5,Paris,48.853495,2.348391
6,Amiens,49.894171,2.295695
7,Lille,50.636565,3.063528
8,Strasbourg,48.584614,7.750713
9,Chateau-du-Haut-Koenigsbourg,48.249523,7.345492


In [5]:
import pandas as pd
import plotly.express as px
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        mapbox_style='open-street-map',
                        hover_name='city',
                        height=600)

fig.show()

## Weather API

In [7]:
os.chdir('../../')

with open("API_key.txt", "r") as file:
    # Read one line from the file
    API_key = str(file.readline())
part = 'current,minutely,hourly,alerts'
weather_cities = []
for city in enriched_cities:
    lat = city['lat']
    lon = city['lon']
    # part=''
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&exclude={part}&appid={API_key}&units=metric'
    weather_raw = requests.get(url).json()
    weather_cities.append(weather_raw)

display(weather_cities)

[{'cod': '200',
  'message': 0,
  'cnt': 40,
  'list': [{'dt': 1711983600,
    'main': {'temp': 13.25,
     'feels_like': 12.54,
     'temp_min': 12.8,
     'temp_max': 13.25,
     'pressure': 997,
     'sea_level': 997,
     'grnd_level': 993,
     'humidity': 73,
     'temp_kf': 0.45},
    'weather': [{'id': 500,
      'main': 'Rain',
      'description': 'light rain',
      'icon': '10d'}],
    'clouds': {'all': 47},
    'wind': {'speed': 7.67, 'deg': 212, 'gust': 10.72},
    'visibility': 10000,
    'pop': 1,
    'rain': {'3h': 0.94},
    'sys': {'pod': 'd'},
    'dt_txt': '2024-04-01 15:00:00'},
   {'dt': 1711994400,
    'main': {'temp': 11.22,
     'feels_like': 10.54,
     'temp_min': 10.09,
     'temp_max': 11.22,
     'pressure': 999,
     'sea_level': 999,
     'grnd_level': 995,
     'humidity': 82,
     'temp_kf': 1.13},
    'weather': [{'id': 500,
      'main': 'Rain',
      'description': 'light rain',
      'icon': '10d'}],
    'clouds': {'all': 56},
    'wind': {'speed'

In [ ]:
pd.json_normalize(weather_cities).head()

,cod,message,cnt,list,city.id,city.name,city.coord.lat,city.coord.lon,city.country,city.population,city.timezone,city.sunrise,city.sunset
0,200,0,40,"[{'dt': 1711962000, 'main': {'temp': 11.28, 'f...",6435453,Huisnes-sur-Mer,48.6360,-1.5115,FR,0,7200,1711950186,1711996572
1,200,0,40,"[{'dt': 1711962000, 'main': {'temp': 11.07, 'f...",2978640,St-Malo,48.6495,-2.0260,FR,50676,7200,1711950308,1711996696
2,200,0,40,"[{'dt': 1711962000, 'main': {'temp': 11.14, 'f...",6447202,Bayeux,49.2765,-0.7025,FR,13911,7200,1711949958,1711996411
3,200,0,40,"[{'dt': 1711962000, 'main': {'temp': 9.51, 'fe...",3003796,Le Havre,49.4939,0.1080,FR,185972,7200,1711949752,1711996228
4,200,0,40,"[{'dt': 1711962000, 'main': {'temp': 9.73, 'fe...",6454707,Rouen,49.4405,1.0940,FR,108569,7200,1711949519,1711995988


In [ ]:
pd.json_normalize(weather_cities)['city.name']
pd.json_normalize(weather_cities)['list'][34] #all data for the city 34 in 'list'
pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])
pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])['weather'][0]

[{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}]

48.636

In [45]:

combined_df = pd.DataFrame()

for id, city in enumerate(pd.json_normalize(weather_cities)['city.name']):

    df3 = pd.json_normalize(pd.json_normalize(weather_cities)['list'][id])
    df3['city'] = city
    df3['lat'] = pd.json_normalize(weather_cities)["city.coord.lat"][id]
    df3['lon'] = pd.json_normalize(weather_cities)["city.coord.lon"][id]
    
    combined_df = pd.concat([combined_df, df3])

combined_df['dt_txt'] = pd.to_datetime(combined_df['dt_txt']) 
display(combined_df)
# display(combined_df.groupby('city').describe().reset_index())


,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.3h,sys.pod,city,lat,lon
0,1711983600,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-01 15:00:00,13.25,12.54,12.80,13.25,997,...,0.45,47,7.67,212,10.72,0.94,d,Huisnes-sur-Mer,48.6360,-1.5115
1,1711994400,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-01 18:00:00,11.22,10.54,10.09,11.22,999,...,1.13,56,8.72,224,14.27,1.68,d,Huisnes-sur-Mer,48.6360,-1.5115
2,1712005200,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-01 21:00:00,9.42,5.78,9.42,9.42,1002,...,0.00,98,8.53,219,13.32,NaN,n,Huisnes-sur-Mer,48.6360,-1.5115
3,1712016000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-02 00:00:00,9.59,5.94,9.59,9.59,1004,...,0.00,96,8.81,236,13.73,NaN,n,Huisnes-sur-Mer,48.6360,-1.5115
4,1712026800,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-02 03:00:00,8.09,4.25,8.09,8.09,1006,...,0.00,57,7.80,226,13.95,NaN,n,Huisnes-sur-Mer,48.6360,-1.5115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,1712361600,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-06 00:00:00,16.01,15.29,16.01,16.01,1000,...,0.00,89,13.90,145,23.61,NaN,n,La Rochelle,46.1591,-1.1520
36,1712372400,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-06 03:00:00,15.64,15.04,15.64,15.64,1004,...,0.00,100,11.27,207,17.25,NaN,n,La Rochelle,46.1591,-1.1520
37,1712383200,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-06 06:00:00,12.75,12.35,12.75,12.75,1007,...,0.00,100,3.27,206,3.92,NaN,d,La Rochelle,46.1591,-1.1520
38,1712394000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-06 09:00:00,13.68,13.11,13.68,13.68,1009,...,0.00,100,2.58,194,3.08,NaN,d,La Rochelle,46.1591,-1.1520


In [46]:
combined_df.head(2)


,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.3h,sys.pod,city,lat,lon
0,1711983600,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-01 15:00:00,13.25,12.54,12.80,13.25,997,...,0.45,47,7.67,212,10.72,0.94,d,Huisnes-sur-Mer,48.636,-1.5115
1,1711994400,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-01 18:00:00,11.22,10.54,10.09,11.22,999,...,1.13,56,8.72,224,14.27,1.68,d,Huisnes-sur-Mer,48.636,-1.5115


In [48]:
col_drop = ['dt', 'weather', 'visibility', 'main.temp', 'main.temp_min', 'main.temp_max', 'main.pressure','main.sea_level',
            'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all', 'wind.deg', 'wind.gust', 'rain.3h']

combined_df.drop(columns=col_drop, inplace=True)


In [49]:
#select days weather
weather_day = combined_df[combined_df['sys.pod']=='d'].drop(columns='sys.pod')

In [50]:
weather_day_mean = weather_day.groupby('city').mean().reset_index()

In [54]:
weather_day_mean.sort_values('pop',ascending=False).head()
weather_day_mean.sort_values('main.feels_like',ascending=False).head()
weather_day_mean.sort_values('wind.speed',ascending=False).head()

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon
19,La Rochelle,0.3664,2024-04-04 02:24:00,12.8340,7.7592,46.1591,-1.1520
30,St-Malo,0.3304,2024-04-04 02:24:00,11.3332,7.1776,48.6495,-2.0260
21,Le Havre,0.4344,2024-04-04 02:24:00,10.7592,7.1092,49.4939,0.1080
7,Bayeux,0.4184,2024-04-04 02:24:00,11.5968,7.0308,49.2765,-0.7025
2,Amiens,0.4744,2024-04-04 02:24:00,12.2068,6.6196,49.8942,2.2957


Create a SCORE to select the city

In [ ]:
weather_day_mean.sort_values('pop',ascending=False)
weather_day_mean.sort_values('main.feels_like',ascending=False)
weather_day_mean.sort_values('wind.speed',ascending=False)

In [60]:

# Assign weights to each metric
weights = {'pop': -0.4, 'main.feels_like': 0.5, 'wind.speed': -0.2}

# Calculate the weighted score for each city
weather_day_mean['score'] = (weather_day_mean['pop'] * weights['pop'] +
                             weather_day_mean['main.feels_like'] * weights['main.feels_like'] +
                             weather_day_mean['wind.speed'] * weights['wind.speed'])

# Normalize the score to a rate between 0 and 10
min_score = weather_day_mean['score'].min()
max_score = weather_day_mean['score'].max()
weather_day_mean['rate'] = 10 * (weather_day_mean['score'] - min_score) / (max_score - min_score)

# display(weather_day_mean.sort_values('rate',ascending=False))
display(weather_day_mean.sort_values('rate',ascending=False).head())

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score,rate
31,Tarascon-sur-Ariège,0.1288,2024-04-04 02:24:00,15.3976,2.4156,42.8458,1.6068,7.16416,10.000000
14,Collioure,0.0020,2024-04-04 02:24:00,15.8624,3.8892,42.5251,3.0832,7.15256,9.968884
8,Bayonne,0.0992,2024-04-04 02:24:00,15.2804,2.5300,43.4945,-1.4737,7.09452,9.813199
10,Biarritz,0.1044,2024-04-04 02:24:00,14.8576,2.7624,43.4833,-1.5593,6.83456,9.115889
5,Avignon,0.0808,2024-04-04 02:24:00,14.8304,3.0684,43.9492,4.8059,6.76920,8.940569
